In [1]:
from torch.utils.tensorboard import SummaryWriter
import os
import json
import pandas as pd


In [2]:
# read the experirements, loop one to scan all hyperparameter values
# params.json store the config (hyperpara)
# progress.csv contain metric for every ep
def gen_tb_report_from_ray(ray_exp_root_folder:str, output_folder: str, metrics_key_list: list, hparam_default_dict: dict = {}):
    
    ########## 1. Find out all Possible Hyper Parameter Keys ##########
    hparam_keys = []
    exps_folder = []

    for filename in os.listdir(ray_exp_root_folder):
        exp_folder = f"{ray_exp_root_folder}/{filename}"
        if filename.startswith(".") or os.path.isfile(exp_folder):
            continue

        exps_folder.append(exp_folder)
        params_fp = f"{exp_folder}/params.json"
        with open(params_fp) as json_file:
            params = json.load(json_file)
            hparam_keys = list(set(hparam_keys + list(params.keys())))

    print("hparam_keys", hparam_keys)
    ########## End 1. Find out all Possible Hyper Parameter Keys ##########


    ########## 2. Write To TensorBoard Scaler, Hparam ##########
    # ref: https://pytorch.org/docs/stable/tensorboard.html
    # for each experiments
    for e, exp_folder in enumerate(exps_folder):
        exp_name = f"exp_{e}"
        params_fp = f"{exp_folder}/params.json"
        with open(params_fp) as json_file:
            params = json.load(json_file)
            for k, v in params.items():
                exp_name += f"_{k}={v}"

        w = SummaryWriter(log_dir=f"{output_folder}/{exp_name}")
        ######### Scalars Report #########
        csv_fp = f"{exp_folder}/progress.csv"
        if os.path.getsize(csv_fp) == 0:
            print("Empty file, skip it")
            continue
        df = pd.read_csv(csv_fp)
        for i, row in df.iterrows():
            for k in metrics_key_list:
                if k in df:
                    w.add_scalar(f"metrics/{k}",row[k], i)
        ######### End Scalars Report #########


        ######### Hyper Parameter Report #########
        #### Build the Hparam Dict
        hparam_dict = {}
        for k in hparam_keys:
            default = ""
            if k in hparam_default_dict:
                default = hparam_default_dict[k]

            params_fp = f"{exp_folder}/params.json"
            with open(params_fp) as json_file:
                params = json.load(json_file)

                hparam_dict[k] = default if k not in params else params[k]

        # read the best matric
        csv_fp = f"{exp_folder}/progress.csv"
        print("csv: ", csv_fp)
        df = pd.read_csv(csv_fp)
        best_it = df.iloc[df['prec1'].argmax()]

        metric_dict = {}
        for k in metrics_key_list:
            if k in df:
                metric_dict[f"best/{k}"] = best_it[k]
        w.add_hparams(hparam_dict, metric_dict)
        ######### End of Hyper Parameter Report #########
        w.close()
    ########## 2. Write To TensorBoard Scaler, Hparam ##########
    
ray_exp_root_folder = "ray_results/run_report"
metrics_key_list = ["lr", "prec1", "train_loss", "val_loss"]
hparam_default_dict = {"arch": "", 
                       "remark": "", 
                       "loss": "ce", 
                       "lr_decay": "step", 
                       "step": -1, 
                       "lr": -1}
gen_tb_report_from_ray(ray_exp_root_folder, "test_tb", metrics_key_list, hparam_default_dict)

hparam_keys ['remark', 'lr_decay', 'arch', 'step', 'loss', 'lr']
csv:  ray_results/run_report/main_0_arch=resnet50,loss=ce,lr=0.01,lr_decay=linear,remark=resume_linear_2020-10-17_16-50-33zggi5r64/progress.csv
csv:  ray_results/run_report/main_0_arch=resnet50,loss=ce,lr=0.05,lr_decay=linear,remark=resume_linear_2020-10-17_13-23-20o7esys9e/progress.csv
csv:  ray_results/run_report/main_0_arch=resnet50,loss=ce,lr=0.05,lr_decay=linear_2020-10-16_20-51-487e84em1z/progress.csv
csv:  ray_results/run_report/main_0_arch=resnet50,loss=focalloss,lr=0.001,lr_decay=cos,remark=resume_step5_2020-10-16_15-06-184vrdo720/progress.csv
csv:  ray_results/run_report/main_0_arch=resnet50,loss=focalloss,lr=0.01,lr_decay=cos_2020-10-16_10-49-54mn5i5pdz/progress.csv
csv:  ray_results/run_report/main_0_arch=resnet50,loss=focalloss,lr=0.1,lr_decay=cos_2020-10-16_12-44-17l49bf821/progress.csv
csv:  ray_results/run_report/main_0_arch=resnet50,lr=0.01,lr_decay=cos_2020-10-16_00-35-57ola4ylls/progress.csv
csv:  ray_r